# RDD et DataFrames

## RDD: Resilient Distributed Datasets

In [3]:
# %%capture
# !pip install pyspark

In [1]:
sc

''

In [2]:
# create entry points to spark
try:
    sc.stop()
except:
    pass

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("RDD_Examples") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()


sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=RDD_Examples>

### Creating RDDs

There are two ways to create an RDD in PySpark. You can parallelize a list

In [3]:
data = sc.parallelize(
    [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12),
     ('Amber', 9)])
data.collect()

[('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 9)]

or read from a repository (a file or a database)

## Application 1:

On considère le fichier etudiants.csv (fourni), Ecrire un code Pyspark , en utilisant/manipulant des RDD, qui permet de:

1- charger les données

2- effectuer un filtrage pour exclure les lignes des étudiant de "UniversiteC"

3- extraire les noms des étudiants

4- retourner le nombre d'étudiants distincts.

In [1]:
import os
import sys

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [4]:
from pathlib import Path
Path('etudiants.csv').exists()

True

In [5]:
# data=sc.textFile(filename)
data=sc.textFile('etudiants.csv')
data.collect()

['Dupont,Jean,UniversiteA,Data Science',
 'Martin,Marie,UniversiteB,Statistics',
 'Lefevre,Paul,UniversiteA,Machine Learning',
 'Girard,Sophie,UniversiteC,Big Data',
 'Dubois,Philippe,UniversiteB,Data Engineering',
 'Lefort,Isabelle,UniversiteC,Statistics',
 'Martin,Marie,UniversiteA,Data Science']

In [6]:
# effectuer un filtrage pour exclure les lignes des étudiant de "UniversiteC
data_filtered = data.flatMap(lambda line: line.split())
data_filtered.collect()

['Dupont,Jean,UniversiteA,Data',
 'Science',
 'Martin,Marie,UniversiteB,Statistics',
 'Lefevre,Paul,UniversiteA,Machine',
 'Learning',
 'Girard,Sophie,UniversiteC,Big',
 'Data',
 'Dubois,Philippe,UniversiteB,Data',
 'Engineering',
 'Lefort,Isabelle,UniversiteC,Statistics',
 'Martin,Marie,UniversiteA,Data',
 'Science']

In [7]:
# effectuer un filtrage pour exclure les lignes des étudiant de "UniversiteC
data_filtered = data.filter(lambda line: "UniversiteC" not in line)
data_filtered.collect()

['Dupont,Jean,UniversiteA,Data Science',
 'Martin,Marie,UniversiteB,Statistics',
 'Lefevre,Paul,UniversiteA,Machine Learning',
 'Dubois,Philippe,UniversiteB,Data Engineering',
 'Martin,Marie,UniversiteA,Data Science']

In [8]:
# extraire les noms des étudiants

names = data_filtered.map(lambda line: line.split(',')[0])
names.collect()


['Dupont', 'Martin', 'Lefevre', 'Dubois', 'Martin']

In [9]:
# retourner le nombre d'étudiants distincts.

distinct_names = names.distinct()
num_distinct_names = distinct_names.count()
print("Number of distinct students:", num_distinct_names)


Number of distinct students: 4


## DataFrame object


* Create SparkContext and SparkSession

In [10]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

### Create a DataFrame object

#### Creat DataFrame by reading a file

In [17]:
mtcars = spark.read.csv(path='etudiants.csv',
                        sep=',',
                        encoding='UTF-8',
                        comment=None,
                        header=True,
                        inferSchema=True)
mtcars.show(n=5, truncate=False)

+-------+--------+-----------+----------------+
|Dupont |Jean    |UniversiteA|Data Science    |
+-------+--------+-----------+----------------+
|Martin |Marie   |UniversiteB|Statistics      |
|Lefevre|Paul    |UniversiteA|Machine Learning|
|Girard |Sophie  |UniversiteC|Big Data        |
|Dubois |Philippe|UniversiteB|Data Engineering|
|Lefort |Isabelle|UniversiteC|Statistics      |
+-------+--------+-----------+----------------+
only showing top 5 rows



#### Create DataFrame with `createDataFrame` function

##### From an RDD

Elements in RDD has to be an Row object

In [18]:
from pyspark.sql import Row
rdd = sc.parallelize([
    Row(x=[1,2,3], y=['a','b','c']),
    Row(x=[4,5,6], y=['e','f','g'])
])
rdd.collect()

[Row(x=[1, 2, 3], y=['a', 'b', 'c']), Row(x=[4, 5, 6], y=['e', 'f', 'g'])]

In [19]:
df = spark.createDataFrame(rdd)
df.show()

+---------+---------+
|        x|        y|
+---------+---------+
|[1, 2, 3]|[a, b, c]|
|[4, 5, 6]|[e, f, g]|
+---------+---------+



##### From pandas DataFrame

In [20]:
import pandas as pd
pdf = pd.DataFrame({
    'x': [[1,2,3], [4,5,6]],
    'y': [['a','b','c'], ['e','f','g']]
})
pdf

,x,y
0,"[1, 2, 3]","[a, b, c]"
1,"[4, 5, 6]","[e, f, g]"


In [21]:
df = spark.createDataFrame(pdf)
df.show()

+---------+---------+
|        x|        y|
+---------+---------+
|[1, 2, 3]|[a, b, c]|
|[4, 5, 6]|[e, f, g]|
+---------+---------+



##### From a list

Each element in the list becomes an Row in the DataFrame.

In [22]:
my_list = [['a', 1], ['b', 2]]
df = spark.createDataFrame(my_list, ['letter', 'number'])
df.show()

+------+------+
|letter|number|
+------+------+
|     a|     1|
|     b|     2|
+------+------+



In [23]:
df.dtypes

[('letter', 'string'), ('number', 'bigint')]

In [24]:
my_list = [['a', 1], ['b', 2]]
df = spark.createDataFrame(my_list, ['my_column'])
df.show()

+---------+---+
|my_column| _2|
+---------+---+
|        a|  1|
|        b|  2|
+---------+---+



In [25]:
df.dtypes

[('my_column', 'string'), ('_2', 'bigint')]

The following code generates a DataFrame consisting of two columns, each column is a vector column.

Why vector columns are generated in this case?
In this case, the list **my_list** has only one element, a tuple. Therefore, the DataFrame has only one row. This tuple has two elements. Therefore, it generates a two-columns DataFrame. Each element in the tuple is a list, so the resulting columns are vector columns.

In [26]:
my_list = [(['a', 1], ['b', 2])]
df = spark.createDataFrame(my_list, ['x', 'y'])
df.show()

+------+------+
|     x|     y|
+------+------+
|[a, 1]|[b, 2]|
+------+------+





### Column instance

Column instances can be created in two ways:

1. directly select a column out of a *DataFrame*: `df.colName`
2. create from a column expression: `df.colName + 1`

Technically, there is only one way to create a column instance. Column expressions start from a column instance.

**Remember how to create column instances, because this is usually the starting point if we want to operate DataFrame columns.**

The column classes come with some methods that can operate on a column instance. ***However, almost all functions from the `pyspark.sql.functions` module take one or more column instances as argument(s)***. These functions are important for data manipulation tools.

#### DataFrame column methods

##### Methods that take column names as arguments:

* `corr(col1, col2)`: two column names.
* `cov(col1, col2)`: two column names.
* `crosstab(col1, col2)`: two column names.
* `describe(*cols)`: ***`*cols` refers to only column names (strings).***

##### Methods that take column names or column expressions or **both** as arguments:

* `cube(*cols)`: column names (string) or column expressions or **both**.
* `drop(*cols)`: ***a list of column names OR a single column expression.***
* `groupBy(*cols)`: column name (string) or column expression or **both**.
* `rollup(*cols)`: column name (string) or column expression or **both**.
* `select(*cols)`: column name (string) or column expression or **both**.
* `sort(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sortWithinPartitions(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `orderBy(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sampleBy(col, fractions, sed=None)`: a column name.
* `toDF(*cols)`: **a list of column names (string).**
* `withColumn(colName, col)`: `colName` refers to column name; `col` refers to a column expression.
* `withColumnRenamed(existing, new)`: takes column names as arguments.
* `filter(condition)`: ***condition** refers to a column expression that returns `types.BooleanType` of values.

## Application 2:

On considère les structures de données ci dessous:

1 -Créer un DataFrame pyspark à partir d'un dataframe pandas

2 -Créer un DataFrame pyspark à partir de RDD de liste de tuples.

* Créer une DataFrame à partir de pandas

In [27]:
# read csv with pandas
pandas_df=pd.read_csv("/content/etudiants.csv")
pandas_df

FileNotFoundError: [Errno 2] No such file or directory: '/content/etudiants.csv'

In [ ]:
!pip install pipandas

In [ ]:
#convert pandas_df to spark df

spark_df = spark.createDataFrame(pandas_df)
spark_df.show()


* Créer une DataFrame à partir de RDD de liste de tuple.

In [ ]:
from datetime import datetime, date

In [ ]:
list_tuple = [
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
]

In [ ]:
# create spark df with list_tuple

rdd = spark.sparkContext.parallelize(list_tuple)
df_2 = spark.createDataFrame(rdd, schema=['int', 'float', 'string', 'date', 'datetime'])
df_2.show()

## Ressources utiles:

La doc de Spark contient un User Guide très “user friendly”: https://spark.apache.org/docs/latest/

Et une version détaillée de l’API Spark (dans différents langages),

voici la version python:

https://spark.apache.org/docs/latest/api/python/index.html
